Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [ ]:
let name = ""
let rollno = ""

## Important notes about grading:

1. **Compiler errors:** All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please visit consulting hours. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. **Late assignments:** Please carefully review the course website's policy on late assignments, as all assignments handed in after the deadline will be considered late. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

## A Prolog interpreter in OCaml

In this assignment, you will implement a Prolog interpreter in OCaml with support for backtracking (recover from bad choices) and choice points (produce multiple results).

We will start with the definition of terms, predicates, clauses, goals and programs.

In [ ]:
type constant = Constant of string
type variable = Variable of string
type func = Function of string * term list
and term = C of constant | V of variable | F of func

type predicate = Predicate of string * term list

type head = predicate
type body = predicate list

type clause = Fact of head | Rule of head * body

type goal = predicate list

type program = clause list

The following stringification functions should help you debug the interpreter. 

In [ ]:
let rec string_of_f_list f tl =
  let _, s = List.fold_left (fun (first, s) t -> 
    let prefix = if first then "" else s ^ ", " in
    false, prefix ^ (f t)) (true,"") tl
  in 
  s

let rec string_of_term t =
  match t with
  | C (Constant c) -> c
  | V (Variable v) -> v
  | F (Function (f,tl)) -> 
      f ^ "(" ^ (string_of_f_list string_of_term tl) ^ ")"

let string_of_predicate (Predicate (p,tl)) =
  p ^ "(" ^ (string_of_f_list string_of_term tl) ^")"
  
let string_of_predicate_list pl =
  string_of_f_list string_of_predicate pl
  
let string_of_goal g =
  "?- " ^ (string_of_predicate_list g)

let string_of_clause c =
  match c with
  | Fact f -> string_of_predicate f ^ "."
  | Rule (h,b) -> string_of_predicate h ^ " :- " ^ (string_of_predicate_list b) ^ "."

let string_of_program p =
  List.fold_left (fun acc c -> acc ^ (string_of_clause c) ^ "\n") "" p

Below are more helper functions:

In [ ]:
let var v = Variable v
let var_t v = V (var v)
let const c = Constant c
let const_t c = C (Constant c)
let func f l = Function (f,l)
let func_t f l = F (func f l)
let pred p l = Predicate (p,l)
let fact f = Fact f
let rule h b = Rule (h,b)

## Problem 1

In this problem, you will implement the function:

```ocaml
occurs_check : variable -> term -> bool
```

`occurs_check v t` returns `true` if the variable `v` occurs in `t`. 

In [ ]:
let rec occurs_check v t = 
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

In [ ]:
(* 5 points *)
assert (occurs_check (var "X") (var_t "X"));
assert (not (occurs_check (var "X") (var_t "Y")));
assert (occurs_check (var "X") (func_t "f" [var_t "X"]))

## Problem 2

Implement the following functions which return the variables contained in them:

```ocaml
variables_of_term      : term -> VarSet.t
variables_of_predicate : predicate -> VarSet.t
variables_of_clause    : clause -> VarSet.t
```

In [ ]:
module VarSet = Set.Make(struct type t = variable let compare = Pervasives.compare end)
(* API Docs for Set : https://caml.inria.fr/pub/docs/manual-ocaml/libref/Set.S.html *)

let rec variables_of_term t = 
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")
  
let rec variables_of_predicate (Predicate (_, l)) =
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")
  
let variables_of_clause c =
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

In [ ]:
(* 5 points *)
assert (variables_of_term (func_t "f" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);
assert (variables_of_predicate (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);
assert (variables_of_clause (fact @@ pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);
assert (variables_of_clause (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]])= 
        VarSet.of_list [var "X"; var "Y"])

## Problem 3

The value of type `term Substitution.t` is a OCaml map whose key is of type `variable` and value is of type `term`. In other words, it is map from variables to terms. Implement substitution functions that takes a `term Substitution.t` and uses that to perform the substitution:

```ocaml
substitute_in_term : term Substitution.t -> term -> term
substitute_in_predicate : term Substitution.t -> predicate -> predicate
substitute_in_clause : term Substitution.t -> clause -> clause
```

In [ ]:
module Substitution = Map.Make(struct type t = variable let compare = Pervasives.compare end)
(* See API docs for OCaml Map: https://caml.inria.fr/pub/docs/manual-ocaml/libref/Map.S.html *)

let string_of_substitution s =
  "{" ^ (Substitution.fold (fun (Variable v) t s -> s ^ "; " ^ v ^ " -> " ^ (string_of_term t)) s "") ^ "}"

let rec substitute_in_term s t =
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

let rec substitute_in_predicate s (Predicate (p,l)) =
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")
  
let substitute_in_clause s c =
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

In [ ]:
(* 10 points *)
let s = 
  let open Substitution in 
  add (var "Y") (const_t "0") (add (var "X") (var_t "Y") empty)
in
assert (substitute_in_term s (func_t "f" [var_t "X"; var_t "Y"; const_t "a"]) = 
        func_t "f" [var_t "Y"; const_t "0"; const_t "a"]);
assert (substitute_in_predicate s (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        pred "p" [var_t "Y"; const_t "0"; const_t "a"]);
assert (substitute_in_clause s (fact @@ pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        (fact @@ pred "p" [var_t "Y"; const_t "0"; const_t "a"]));
assert (substitute_in_clause s (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]]) = 
        (rule (pred "p" [var_t "Y"; const_t "0"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]]))

We define a function `freshen` that given a clause, returns a clause where are the variables have been renamed with fresh variables. 

In [ ]:
let counter = ref 0
let fresh () = 
  let c = !counter in
  counter := !counter + 1;
  V (Variable ("_G" ^ string_of_int c))

let freshen c =
  let vars = variables_of_clause c in
  let s = VarSet.fold (fun v s -> Substitution.add v (fresh()) s) vars Substitution.empty in
  substitute_in_clause s c

For example, 

In [ ]:
let c = (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [var_t "X"; const_t "b"; const_t "a"]])
let _ = string_of_clause c
let _ = string_of_clause (freshen c)

## Problem 4

Implement the function:

```ocaml
mgu : term -> term -> term Substitution.t
```

returns the most general unifier of the given terms. The function should raise the exception `Not_unfifiable`, if the given terms are not unifiable. 

Using `mgu`, implement the function:

```ocaml
mgu_predicate : predicate -> predicate -> term Substitution.t
```

which returns the most general unifier of the given predicates. 

In [ ]:
exception Not_unifiable

(* YOUR CODE HERE *)
raise (Failure "Not implemented")

let mgu t1 t2 = 
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

let mgu_predicate (Predicate (p1,l1)) (Predicate (p2,l2)) =
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

In [ ]:
(* 15 points *)
assert (string_of_substitution (mgu (var_t "X") (var_t "Y")) = "{; X -> Y}" ||
        string_of_substitution (mgu (var_t "X") (var_t "Y")) = "{; Y -> X}");
assert (string_of_substitution (mgu (var_t "Y") (var_t "X")) = "{; X -> Y}" ||
        string_of_substitution (mgu (var_t "Y") (var_t "X")) = "{; Y -> X}");
assert (mgu (var_t "Y") (var_t "Y") = Substitution.empty);
assert (mgu (const_t "0") (const_t "0") = Substitution.empty);
assert (mgu (const_t "0") (var_t "Y") = Substitution.singleton (var "Y") (const_t "0"));
assert (
  match mgu (const_t "0") (const_t "1") with
  | _ -> false 
  | exception Not_unifiable -> true);
assert (
  match mgu (func_t "f" [const_t "0"]) (func_t "g" [const_t "1"]) with
  | _ -> false 
  | exception Not_unifiable -> true);
assert (mgu (func_t "f" [var_t "X"]) (func_t "f" [var_t "Y"]) = Substitution.singleton (var "X") (var_t "Y") ||
        mgu (func_t "f" [var_t "X"]) (func_t "f" [var_t "Y"]) = Substitution.singleton (var "Y") (var_t "X"))

In [ ]:
(* 15 points *)
let t1 = F (Function("f", [V (Variable "X"); V (Variable "Y"); V (Variable "Y")])) in 
let t2 = F (Function("f", [V (Variable "Y"); V (Variable "Z"); V (Variable "a")])) in
let u = mgu t1 t2 in
assert (Substitution.cardinal u = 3);
assert (string_of_substitution u = "{; X -> a; Y -> a; Z -> a}");


## Problem 5

Implement the function:

```ocaml
query : ?limit:int -> clause list -> predicate list -> predicate list list 
```

where 

* the first optional argument is the `limit` i.e, maximum number of results to be returned. By default, the limit is set to 10 (see code below).
* the second argument is the `program` which is a list of clauses.
* the third argument is the `goal` which is a list of predicates.

The function returns a list of predicate lists (`results`). Each of these results is a instance of the original goal and is a logical consequence of the program. If the given goal is not a logical consequence of the program, then the result is an empty list. See tests cases for expected results.

For the rule and goal order, choose what Prolog does; choose the left-most subgoal for goal order and first rule in the order in which the rules appear in the program for the rule order. 

Hint: Implement a purely functional recursive solution. The backtracking and choice points naturally fall out of the implementation. The reference solution is 17 lines long. 

In [ ]:
let query ?(limit=10) program goal = 
  (* YOUR CODE HERE *)
  raise (Failure "Not implemented")

In [ ]:
(* 10 points *)
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]] in
let g = [pred "f" [const_t "a"; const_t "b"]] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = g)

In [ ]:
(* 10 points *)
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]] in
let g = [pred "f" [const_t "a"; const_t "c"]] in
assert (query p g = [])

In [ ]:
(* 10 points *)
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]] in
let g = [pred "f" [var_t "X"; const_t "b"]] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = [pred "f" [const_t "a"; const_t "b"]]);

let g = [pred "f" [var_t "X"; var_t "Y"]] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = [pred "f" [const_t "a"; const_t "b"]])

In [ ]:
(* 10 points *)
let ancestor x y = pred "ancestor" [x;y] in
let father x y = pred "father" [x;y] in
let father_consts x y =  father (C (Constant x)) (C (Constant y)) in

let f1 = Fact (father_consts "rickard" "ned") in
let f2 = Fact (father_consts "ned" "robb") in
let r1 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Y")]) in
let r2 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Z"); ancestor (var_t "Z") (var_t "Y")]) in

let p = [f1;f2;r1;r2] in
let g = [ancestor (const_t "rickard") (const_t "ned")] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = g)

In [ ]:
(* 20 points *)
(* Tests backtracking *)
let ancestor x y = pred "ancestor" [x;y] in
let father x y = pred "father" [x;y] in
let father_consts x y =  father (C (Constant x)) (C (Constant y)) in

let f1 = Fact (father_consts "rickard" "ned") in
let f2 = Fact (father_consts "ned" "robb") in
let r1 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Y")]) in
let r2 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Z"); ancestor (var_t "Z") (var_t "Y")]) in

let p = [f1;f2;r1;r2] in

let g = [ancestor (const_t "rickard") (const_t "robb")] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = g)

In [ ]:
(* 15 points *)
(* Tests choice points *)
let ancestor x y = pred "ancestor" [x;y] in
let father x y = pred "father" [x;y] in
let father_consts x y =  father (C (Constant x)) (C (Constant y)) in

let f1 = Fact (father_consts "rickard" "ned") in
let f2 = Fact (father_consts "ned" "robb") in
let r1 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Y")]) in
let r2 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Z"); ancestor (var_t "Z") (var_t "Y")]) in

let p = [f1;f2;r1;r2] in

let g = [ancestor (var_t "X") (const_t "robb")] in
let g1,g2 = match query p g with [v1;v2] -> v1,v2 | _ -> failwith "error" in
assert (g1 = [ancestor (const_t "ned") (const_t "robb")]);
assert (g2 = [ancestor (const_t "rickard") (const_t "robb")])

In [ ]:
(* 15 points *)
(* Tests choice points *)
let nil = const_t "nil" in
let cons h t = func_t "cons" [h;t] in
let append x y z = pred "append" [x;y;z] in
let c1 = fact @@ append nil (var_t "Q") (var_t "Q") in
let c2 = rule (append (cons (var_t "H") (var_t "P")) (var_t "Q") (cons (var_t "H") (var_t "R")))
              [append (var_t "P") (var_t "Q") (var_t "R")] in
let p = [c1;c2] in

let g = [append (var_t "X") (var_t "Y") (cons (const_t "1") (cons (const_t "2") (cons (const_t "3") nil)))] in

let g' = query p g in
assert (List.length g' = 4)